<a href="https://colab.research.google.com/github/atick-faisal/Crowd-Emotion/blob/main/src_v4/CE_TE_MobileNet_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import joblib
import numpy as np
import tensorflow as tf

from sklearn.utils import shuffle
from sklearn.metrics import classification_report, accuracy_score

tf.__version__

'2.4.1'

In [2]:
BASE_DIR         = '/content/drive/MyDrive/Research/Crowd Emotion v4/'
DATASET_DIR      = 'DatasetPC/'
FOLDS            = ['Fold 1', 'Fold 2', 'Fold 3', 'Fold 4', 'Fold 5']
EMOTIONS         = ['Approval', 'Disapproval', 'Neutral']

IMG_LEN          = 160
IMG_SIZE         = (IMG_LEN, IMG_LEN)
DATASET_LEN      = 14882

CONFIG           = 'LOG_SPEC_W400_H200_HAMM_IMG' + str(IMG_LEN)

TEST_FOLD        = 'Fold 1'
EPOCHS           = 10
LEARNING_RATE    = 0.001

In [3]:
path = os.path.join(BASE_DIR, DATASET_DIR)
X    = joblib.load(path + CONFIG + '_X.joblib')
y    = joblib.load(path + CONFIG + '_y.joblib')
f    = joblib.load(path + CONFIG + '_f.joblib')

In [4]:
# mask      = (f == FOLDS.index(TEST_FOLD))
# X_train   = X[~mask, :]
# X_test    = X[mask, :]
# y_train   = y[~mask, :]
# y_test    = y[mask, :]

In [5]:
X, y = shuffle(X, y)
X_train     = X[:12000, :]
y_train     = y[:12000, :]
X_test      = X[12000:, :]
y_test      = y[12000:, :]

In [6]:
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input
rescale = tf.keras.layers.experimental.preprocessing.Rescaling(1./127.5, offset= -1)

In [7]:
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')
base_model.trainable = False

In [8]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(len(EMOTIONS))

In [9]:
def get_model():
    inputs = tf.keras.Input(shape=IMG_SHAPE)
    x = preprocess_input(inputs)
    x = base_model(x, training=False)
    x = global_average_layer(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    outputs = prediction_layer(x)
    model = tf.keras.Model(inputs, outputs)
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=LEARNING_RATE),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
    return model

In [10]:
model = get_model()
# X_train, y_train = shuffle(X_train, y_train)
history = model.fit(X_train, y_train, epochs=EPOCHS)

Epoch 1/10
375/375 [==============================] - 15s 31ms/step - loss: 0.7019 - accuracy: 0.6971
Epoch 2/10
375/375 [==============================] - 11s 31ms/step - loss: 0.3935 - accuracy: 0.8432
Epoch 3/10
375/375 [==============================] - 12s 31ms/step - loss: 0.3269 - accuracy: 0.8732
Epoch 4/10
375/375 [==============================] - 12s 31ms/step - loss: 0.3106 - accuracy: 0.8746
Epoch 5/10
375/375 [==============================] - 11s 31ms/step - loss: 0.3032 - accuracy: 0.8837
Epoch 6/10
375/375 [==============================] - 11s 31ms/step - loss: 0.2886 - accuracy: 0.8877
Epoch 7/10
375/375 [==============================] - 12s 31ms/step - loss: 0.2720 - accuracy: 0.8906
Epoch 8/10
375/375 [==============================] - 12s 31ms/step - loss: 0.2677 - accuracy: 0.8939
Epoch 9/10
375/375 [==============================] - 12s 31ms/step - loss: 0.2564 - accuracy: 0.8930
Epoch 10/10
375/375 [==============================] - 12s 31ms/step - loss: 0.252

In [11]:
prob = tf.keras.Sequential([model, tf.keras.layers.Softmax()])
y_pred_hot = prob.predict(X_test)
y_pred = np.argmax(y_pred_hot, axis=1)
print(classification_report(y_test.ravel(), y_pred))

              precision    recall  f1-score   support

           0       0.91      0.89      0.90      1342
           1       0.92      0.89      0.91       379
           2       0.88      0.91      0.90      1161

    accuracy                           0.90      2882
   macro avg       0.90      0.90      0.90      2882
weighted avg       0.90      0.90      0.90      2882

